## We solve the MHD toy model with PINN

### The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) \;\;\;\; \nabla_i B^i = 0
$$

The vector field $v^i$ is given. 

And we impose Dirichlet boundary conditions.

The initial data is: 

\begin{align*}
B_1 &= \partial_y \phi \\
B_2 &= -\partial_x \phi
\end{align*}
with (here we use xmin=0, xmax = 1, same for y)

$$
\phi(x,y) = (x*(x-1)*y*(y-1))^2
$$

The velocity field is time-independent and given by:

\begin{align*}
v1(t,x,y) &= \sin(\pi*x)*\cos(\pi*y) \\
v2(t,x,y) &= \cos(\pi*x)*\sin(\pi*y) \\
\end{align*}



In [ ]:
using NeuralPDE, Optimization, OptimizationOptimJL, Roots,
      LineSearches
using ModelingToolkit, IntervalSets 
using IntervalSets
using Plots, Printf
using Lux, LuxCUDA, ComponentArrays, Random
using JLD2, LinearAlgebra 
const gpud = gpu_device()

@parameters t, x, y
@variables B1(..), B2(..)
Dx = Differential(x)
Dy = Differential(y)
Dt = Differential(t)

In [ ]:
# Space and time domains
@show domains = [t ∈ Interval(0.0, 4.0),
    x ∈ Interval(0.0, 1.0), y ∈ Interval(0.0, 1.0)]
# Discretization
dx = [0.1, 0.1, 0.1]

In [ ]:
v1(t,x,y) = -sin(pi*x)
v2(t,x,y) = -sin(pi*y) # with minus is divergence free

phi(x,y) = (x*(x-1)*y*(y-1))^2*64 # to make it zero at the boundaries




In [ ]:
xs, ys = [0.0:0.01:1.0 for d in domains]

v1_d = [v1(0, x, y) for x in xs for y in ys]
v2_d = [v2(0, x, y) for x in xs for y in ys]
pv1 = plot(xs, ys, v1_d, linetype = :contourf, title = "v1", aspect_ratio = 1)
pv2 = plot(xs, ys, v2_d, linetype = :contourf, title = "v2", aspect_ratio = 1)
plot(pv1, pv2, layout = (1,2))

In [ ]:
B1_d = [ y*(y-1)*(2y-1)*x^2*(x-1)^2*128 for x in xs for y in ys]
B2_d = [ -x*(x-1)*(2x-1)*y^2*(y-1)^2*128 for x in xs for y in ys]
pv1 = plot(xs, ys, B1_d, linetype = :contourf, title = "Initial B1", aspect_ratio = 1)
pv2 = plot(xs, ys, B2_d, linetype = :contourf, title = "Initial B2", aspect_ratio = 1)
plot(pv1, pv2, layout = (1,2))

In [ ]:

eqs = [Dt(B1(t,x,y)) ~ Dy(v2(t,x,y)*B1(t,x,y) - v1(t,x,y)*B2(t,x,y)), 
    Dt(B2(t,x,y)) ~ Dx(v1(t,x,y)*B2(t,x,y) - v2(t,x,y)*B1(t,x,y)), 
    Dx(B1(t,x,y)) + Dy(B2(t,x,y))  ~ 0]


In [ ]:

bcs = [B1(0,x,y) ~ Dy(phi(x,y)),
    B2(0,x,y) ~ -Dx(phi(x,y)),
    B1(t,0,y) ~ 0,
    B1(t,1,y) ~ 0,
    B1(t,x,0) ~ 0,
    B1(t,x,1) ~ 0,
    B2(t,0,y) ~ 0,
    B2(t,1,y) ~ 0,
    B2(t,x,0) ~ 0,
    B2(t,x,1) ~ 0]



In [ ]:
# Neural network
input_ = length(domains)
n = 16
chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:2]

#strategy = QuadratureTraining()
strategy = GridTraining(dx)
#ps = Lux.setup(Random.default_rng(), chain)[1]
#ps = ps |> ComponentArray |> gpud .|> Float64
discretization = PhysicsInformedNN(chain, strategy)

@named MHD_toy = PDESystem(eqs, bcs, domains, [t, x, y], [B1(t, x, y), B2(t, x, y)])
prob = discretize(MHD_toy, discretization)
sym_prob = symbolic_discretize(MHD_toy, discretization)

pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

loss = Float64[]

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    push!(loss, l)
    return false
end

res = Optimization.solve(prob, BFGS(linesearch = BackTracking()); maxiters = 100, callback)


In [ ]:
phi_d = discretization.phi

In [ ]:
using JLD2
@save "toy_MHD_2D.jld2" chain res phi_d loss Error

In [ ]:
xs, ys = [0.0:0.01:1.0 for d in domains]
depvars = [:B1, :B2]
minimizers_ = [res.u.depvar[depvars[i]] for i in 1:length(chain)]

t_max = 8.0
u_predict = [phi_d[i]([t, x, y], minimizers_[i])[1] for x in xs for y in ys for i in 1:2]


In [ ]:
size(u_predict)

In [ ]:

ps = []
for i in 1:2
    p1 = plot(xs, ys, u_predict[:,:,i], linetype = :contourf, title = "predict", aspect_ratio = 1)
    push!(ps, p1)
end

plot(ps[1], ps[2], layout = (1,2))



In [ ]:


function plot_(res)
    # Animate
    anim = @animate for (i, t) in enumerate(0:0.05:t_max)
        @info "Animating frame $i..."
        B1_approx = reshape([Array(phi_d[1]([t, x, y], res.u))[1] for x in xs for y in ys],
            length(xs), length(ys))
        B1_approx = reshape([Array(phi_d[2]([t, x, y], res.u))[1] for x in xs for y in ys],
            length(xs), length(ys))
        title = @sprintf("B1, t = %.3f", t)
        p1 = plot(xs, ys, B1_approx, st = :surface, label = "B1", title = title)
        title = @sprintf("B2, t = %.3f", t)
        p2 = plot(xs, ys, B2_approx, st = :surface, label = "", title = title)
        plot(p1, p2)
    end
    gif(anim, "Toy_MHD_pde.gif", fps = 10)
end

plot_(res)